# SecSSE Size

### Loading packages

In [1]:
# Loading packages
library("mclust")
library("Rtsne")
library("ape")
library("secsse")
library("DDD")
library("doMC")
library("tidyverse")
library("parallel")
library("qgraph")
library("stringr")

Package 'mclust' version 6.0.0
Type 'citation("mclust")' for citing this R package in publications.

Le chargement a nécessité le package : foreach

Le chargement a nécessité le package : iterators

Le chargement a nécessité le package : parallel

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.2 ──
✔ ggplot2 3.4.4     ✔ purrr   1.0.2
✔ tibble  3.2.1     ✔ dplyr   1.1.4
✔ tidyr   1.3.0     ✔ stringr 1.5.1
✔ readr   2.1.3     ✔ forcats 0.5.2
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ purrr::accumulate() masks foreach::accumulate()
✖ dplyr::filter()     masks stats::filter()
✖ dplyr::lag()        masks stats::lag()
✖ purrr::map()        masks mclust::map()
✖ purrr::when()       masks foreach::when()
✖ dplyr::where()      masks ape::where()


### Loading data

In [2]:
# Loading data

df<-read.csv("../../Data2/Species_data.tsv", sep="\t") # omit sep ="\t" for .csv files

phy<-read.nexus("../../Data2/16FC_16C_374_sp.tree")

### Clusterizing data

In [3]:
mb = Mclust(as.numeric(df$Body.size))
summary(mb, parameters = TRUE)
df$Body.size <- as.factor(mb$classification)

---------------------------------------------------- 
Gaussian finite mixture model fitted by EM algorithm 
---------------------------------------------------- 

Mclust V (univariate, unequal variance) model with 3 components: 

 log-likelihood   n df       BIC       ICL
       -3051.42 545  8 -6153.246 -6359.293

Clustering table:
  1   2   3 
279 210  56 

Mixing probabilities:
        1         2         3 
0.4435607 0.4288234 0.1276159 

Means:
        1         2         3 
 56.17492 114.10754 372.12177 

Variances:
         1          2          3 
  349.3196  1723.1476 74082.5716 

### Optional step (cleaning data)

In [4]:
# Optional step (cleaning data)
states<-df$Body.size
names(states)=str_replace((df$Species), " ", "_")
states2<-states[!names(states) %in% setdiff(names(states), phy$tip.label)]

In [5]:
states2<-states[!names(states) %in% setdiff(names(states), phy$tip.label)]

In [6]:
states3<-as.data.frame(cbind(names(states2), as.factor(states2)))
colnames(states3)<-c("species","states")
rownames(states3)<-NULL
traits <- sortingtraits(states3, phy)
traits<-traits-1

### Computing sampling fractions

In [7]:
# Computing sampling fractions
f<-c(
length(states2[states2=="1"])/length(df$Body.size[df$Body.size=="1"]),
length(states2[states2=="2"])/length(df$Body.size[df$Body.size=="2"]),
length(states2[states2=="3"])/length(df$Body.size[df$Body.size=="3"]))

### Starting values

In [8]:
BD_model <- bd_ML(brts = ape::branching.times(phy))
BD_lambda <- BD_model$lambda0
BD_mu <- BD_model$mu0

strt_lambda<-c(BD_lambda, BD_lambda/2, BD_lambda*2)
strt_mu <- c(BD_mu, BD_mu/2, BD_mu*2)
strt_q <- strt_lambda/3

You are optimizing lambda0 mu0 
You are fixing lambda1 mu1 
Optimizing the likelihood - this may take a while. 
The loglikelihood for the initial parameter values is -2000.08885924251.

Maximum likelihood parameter estimates: lambda0: 0.030162, mu0: 0.012378, lambda1: 0.000000, mu1: 0.000000: 
Maximum loglikelihood: -1757.495690


## Constant Rate

In [9]:
spec_matrix <- c()
spec_matrix <- rbind(spec_matrix, c(0, 0, 0, 1))
spec_matrix <- rbind(spec_matrix, c(1, 1, 1, 1))
spec_matrix <- rbind(spec_matrix, c(2, 2, 2, 1))

In [10]:
lambda_list_cr <- secsse::create_lambda_list(state_names = c(0, 1, 2),
                                          num_concealed_states = 3,
                                          transition_matrix = spec_matrix,
                                          model = "CR")
lambda_list_cr

,0A,1A,2A,0B,1B,2B,0C,1C,2C
0A,1,0,0,0,0,0,0,0,0
1A,0,0,0,0,0,0,0,0,0
2A,0,0,0,0,0,0,0,0,0
0B,0,0,0,0,0,0,0,0,0
1B,0,0,0,0,0,0,0,0,0
2B,0,0,0,0,0,0,0,0,0
0C,0,0,0,0,0,0,0,0,0
1C,0,0,0,0,0,0,0,0,0
2C,0,0,0,0,0,0,0,0,0
,0A,1A,2A,0B,1B,2B,0C,1C,2C


In [11]:
mu_vec_cr <- secsse::create_mu_vector(state_names = c(0, 1, 2),
                                   num_concealed_states = 3,
                                   model = "CR",
                                   lambda_list = lambda_list_cr)
mu_vec_cr

0A 1A 2A 0B 1B 2B 0C 1C 2C 
 2  2  2  2  2  2  2  2  2

In [12]:
shift_matrix_cr  <- c()
shift_matrix_cr  <- rbind(shift_matrix_cr , c(0, 1, 3))
shift_matrix_cr  <- rbind(shift_matrix_cr , c(1, 0, 3))
shift_matrix_cr  <- rbind(shift_matrix_cr , c(1, 2, 4))
shift_matrix_cr  <- rbind(shift_matrix_cr , c(2, 1, 4))

In [13]:
q_matrix_cr <- secsse::create_q_matrix(state_names = c(0, 1, 2),
                                    num_concealed_states = 3,
                                    shift_matrix = shift_matrix_cr,
                                    diff.conceal = TRUE)
q_matrix_cr[q_matrix_cr>=5]<-5
q_matrix_cr

,0A,1A,2A,0B,1B,2B,0C,1C,2C
0A,NA,3,0,5,0,0,5,0,0
1A,3,NA,4,0,5,0,0,5,0
2A,0,4,NA,0,0,5,0,0,5
0B,5,0,0,NA,3,0,5,0,0
1B,0,5,0,3,NA,4,0,5,0
2B,0,0,5,0,4,NA,0,0,5
0C,5,0,0,5,0,0,NA,3,0
1C,0,5,0,0,5,0,3,NA,4
2C,0,0,5,0,0,5,0,4,NA


In [14]:
### Try 1

idparsopt <- c(1,2,3,4,5) # our maximum rate parameter was 3
idparsfix <- c(0) # we want to keep all zeros at zero
initparsopt <- c(strt_lambda[1], strt_mu[1], rep(strt_q[1],3))
initparsfix <- c(0.0) # all zeros remain at zero.
idparslist <- list()
idparslist[[1]] <- lambda_list_cr
idparslist[[2]] <- mu_vec_cr
idparslist[[3]] <- q_matrix_cr

model <- secsse::cla_secsse_ml(phy = phy,
                              traits = traits,
                              num_concealed_states = 3,
                              idparslist = idparslist,
                              idparsopt = idparsopt,
                              initparsopt = initparsopt,
                              idparsfix = idparsfix,
                              parsfix = initparsfix,
                              sampling_fraction = f,
                              verbose = FALSE,
                              num_threads =  8)
model$k <- 5
saveRDS(model, "SecSSE_Results/Size_results/size_model_try1.rds")

Warning message in check_ml_conditions(traits, idparslist, initparsopt, idparsopt, :
“Note: you set some transitions as impossible to happen.”


In [15]:
### Try 2

idparsopt <- c(1,2,3,4,5) # our maximum rate parameter was 3
idparsfix <- c(0) # we want to keep all zeros at zero
initparsopt <- c(strt_lambda[2], strt_mu[2], rep(strt_q[2],3))
initparsfix <- c(0.0) # all zeros remain at zero.
idparslist <- list()
idparslist[[1]] <- lambda_list_cr
idparslist[[2]] <- mu_vec_cr
idparslist[[3]] <- q_matrix_cr

model <- secsse::cla_secsse_ml(phy = phy,
                              traits = traits,
                              num_concealed_states = 3,
                              idparslist = idparslist,
                              idparsopt = idparsopt,
                              initparsopt = initparsopt,
                              idparsfix = idparsfix,
                              parsfix = initparsfix,
                              sampling_fraction = f,
                              verbose = FALSE,
                              num_threads =  8)
model$k <- 5
saveRDS(model, "SecSSE_Results/Size_results/size_model_try2.rds")

Warning message in check_ml_conditions(traits, idparslist, initparsopt, idparsopt, :
“Note: you set some transitions as impossible to happen.”


In [16]:
### Try 3

idparsopt <- c(1,2,3,4,5) # our maximum rate parameter was 3
idparsfix <- c(0) # we want to keep all zeros at zero
initparsopt <- c(strt_lambda[3], strt_mu[3], rep(strt_q[3],3))
initparsfix <- c(0.0) # all zeros remain at zero.
idparslist <- list()
idparslist[[1]] <- lambda_list_cr
idparslist[[2]] <- mu_vec_cr
idparslist[[3]] <- q_matrix_cr

model <- secsse::cla_secsse_ml(phy = phy,
                              traits = traits,
                              num_concealed_states = 3,
                              idparslist = idparslist,
                              idparsopt = idparsopt,
                              initparsopt = initparsopt,
                              idparsfix = idparsfix,
                              parsfix = initparsfix,
                              sampling_fraction = f,
                              verbose = FALSE,
                              num_threads =  8)
model$k <- 5
saveRDS(model, "SecSSE_Results/Size_results/size_model_try3.rds")

Warning message in check_ml_conditions(traits, idparslist, initparsopt, idparsopt, :
“Note: you set some transitions as impossible to happen.”


## Examined trait 

In [17]:
spec_matrix <- c()
spec_matrix <- rbind(spec_matrix, c(0, 0, 0, 1))
spec_matrix <- rbind(spec_matrix, c(1, 1, 1, 2))
spec_matrix <- rbind(spec_matrix, c(2, 2, 2, 3))

In [18]:
lambda_list_etd <- secsse::create_lambda_list(state_names = c(0, 1, 2),
                                          num_concealed_states = 3,
                                          transition_matrix = spec_matrix,
                                          model = "ETD")
lambda_list_etd

,0A,1A,2A,0B,1B,2B,0C,1C,2C
0A,1,0,0,0,0,0,0,0,0
1A,0,0,0,0,0,0,0,0,0
2A,0,0,0,0,0,0,0,0,0
0B,0,0,0,0,0,0,0,0,0
1B,0,0,0,0,0,0,0,0,0
2B,0,0,0,0,0,0,0,0,0
0C,0,0,0,0,0,0,0,0,0
1C,0,0,0,0,0,0,0,0,0
2C,0,0,0,0,0,0,0,0,0
,0A,1A,2A,0B,1B,2B,0C,1C,2C


In [19]:
mu_vec_etd <- secsse::create_mu_vector(state_names = c(0, 1, 2),
                                   num_concealed_states = 3,
                                   model = "ETD",
                                   lambda_list = lambda_list_etd)
mu_vec_etd

0A 1A 2A 0B 1B 2B 0C 1C 2C 
 4  5  6  4  5  6  4  5  6

In [20]:
shift_matrix <- c()
shift_matrix <- rbind(shift_matrix, c(0, 1, 7))
shift_matrix <- rbind(shift_matrix, c(1, 0, 7))
shift_matrix <- rbind(shift_matrix, c(1, 2, 8))
shift_matrix <- rbind(shift_matrix, c(2, 1, 8))

In [21]:
q_matrix <- secsse::create_q_matrix(state_names = c(0, 1, 2),
                                    num_concealed_states = 3,
                                    shift_matrix = shift_matrix,
                                    diff.conceal = TRUE)
q_matrix[q_matrix>=9]<-9
q_matrix

,0A,1A,2A,0B,1B,2B,0C,1C,2C
0A,NA,7,0,9,0,0,9,0,0
1A,7,NA,8,0,9,0,0,9,0
2A,0,8,NA,0,0,9,0,0,9
0B,9,0,0,NA,7,0,9,0,0
1B,0,9,0,7,NA,8,0,9,0
2B,0,0,9,0,8,NA,0,0,9
0C,9,0,0,9,0,0,NA,7,0
1C,0,9,0,0,9,0,7,NA,8
2C,0,0,9,0,0,9,0,8,NA


### Speciation rates

In [22]:
mu_vec_etd_sp<-rep(4, length(mu_vec_etd))

In [23]:
### Try 1
idparsopt <- c(1:4, 7:9) # our maximum rate parameter was 3
idparsfix <- c(0) # we want to keep all zeros at zero
initparsopt <- c(rep(strt_lambda[1],3), rep(strt_mu[1],1), rep(strt_q[1],3))
initparsfix <- c(0.0) # all zeros remain at zero.
idparslist <- list()
idparslist[[1]] <- lambda_list_etd
idparslist[[2]] <- mu_vec_etd_sp
idparslist[[3]] <- q_matrix

model <- secsse::cla_secsse_ml(phy = phy,
                              traits = traits,
                              num_concealed_states = 3,
                              idparslist = idparslist,
                              idparsopt = idparsopt,
                              initparsopt = initparsopt,
                              idparsfix = idparsfix,
                              parsfix = initparsfix,
                              sampling_fraction = f,
                              verbose = FALSE,
                              num_threads =  8)
model$k <- 7
saveRDS(model, "SecSSE_Results/Size_results/size_examined_trait_sp_try1.rds")

Warning message in check_ml_conditions(traits, idparslist, initparsopt, idparsopt, :
“Note: you set some transitions as impossible to happen.”


In [24]:
### Try 2
idparsopt <- c(1:4, 7:9) # our maximum rate parameter was 3
idparsfix <- c(0) # we want to keep all zeros at zero
initparsopt <- c(rep(strt_lambda[2],3), rep(strt_mu[2],1), rep(strt_q[2],3))
initparsfix <- c(0.0) # all zeros remain at zero.
idparslist <- list()
idparslist[[1]] <- lambda_list_etd
idparslist[[2]] <- mu_vec_etd_sp
idparslist[[3]] <- q_matrix

model <- secsse::cla_secsse_ml(phy = phy,
                              traits = traits,
                              num_concealed_states = 3,
                              idparslist = idparslist,
                              idparsopt = idparsopt,
                              initparsopt = initparsopt,
                              idparsfix = idparsfix,
                              parsfix = initparsfix,
                              sampling_fraction = f,
                              verbose = FALSE,
                              num_threads =  8)
model$k <- 7
saveRDS(model, "SecSSE_Results/Size_results/size_examined_trait_sp_try2.rds")

Warning message in check_ml_conditions(traits, idparslist, initparsopt, idparsopt, :
“Note: you set some transitions as impossible to happen.”


In [25]:
### Try 3
idparsopt <- c(1:4, 7:9) # our maximum rate parameter was 3
idparsfix <- c(0) # we want to keep all zeros at zero
initparsopt <- c(rep(strt_lambda[3],3), rep(strt_mu[3],1), rep(strt_q[3],3))
initparsfix <- c(0.0) # all zeros remain at zero.
idparslist <- list()
idparslist[[1]] <- lambda_list_etd
idparslist[[2]] <- mu_vec_etd_sp
idparslist[[3]] <- q_matrix

model <- secsse::cla_secsse_ml(phy = phy,
                              traits = traits,
                              num_concealed_states = 3,
                              idparslist = idparslist,
                              idparsopt = idparsopt,
                              initparsopt = initparsopt,
                              idparsfix = idparsfix,
                              parsfix = initparsfix,
                              sampling_fraction = f,
                              verbose = FALSE,
                              num_threads =  8)
model$k <- 7
saveRDS(model, "SecSSE_Results/Size_results/size_examined_trait_sp_try3.rds")

Warning message in check_ml_conditions(traits, idparslist, initparsopt, idparsopt, :
“Note: you set some transitions as impossible to happen.”


### Extinction rates

In [26]:
### Try 1
idparsopt <- c(1, 4:9) # our maximum rate parameter was 3
idparsfix <- c(0) # we want to keep all zeros at zero
initparsopt <- c(rep(strt_lambda[1],1), rep(strt_mu[1],3), rep(strt_q[1],3))
initparsfix <- c(0.0) # all zeros remain at zero.
idparslist <- list()
idparslist[[1]] <- lambda_list_cr
idparslist[[2]] <- mu_vec_etd
idparslist[[3]] <- q_matrix

model <- secsse::cla_secsse_ml(phy = phy,
                              traits = traits,
                              num_concealed_states = 3,
                              idparslist = idparslist,
                              idparsopt = idparsopt,
                              initparsopt = initparsopt,
                              idparsfix = idparsfix,
                              parsfix = initparsfix,
                              sampling_fraction = f,
                              verbose = FALSE,
                              num_threads =  8)
model$k <- 7
saveRDS(model, "SecSSE_Results/Size_results/size_examined_trait_mu_try1.rds")

Warning message in check_ml_conditions(traits, idparslist, initparsopt, idparsopt, :
“Note: you set some transitions as impossible to happen.”


In [27]:
### Try 2
idparsopt <- c(1, 4:9) # our maximum rate parameter was 3
idparsfix <- c(0) # we want to keep all zeros at zero
initparsopt <- c(rep(strt_lambda[2],1), rep(strt_mu[2],3), rep(strt_q[2],3))
initparsfix <- c(0.0) # all zeros remain at zero.
idparslist <- list()
idparslist[[1]] <- lambda_list_cr
idparslist[[2]] <- mu_vec_etd
idparslist[[3]] <- q_matrix

model <- secsse::cla_secsse_ml(phy = phy,
                              traits = traits,
                              num_concealed_states = 3,
                              idparslist = idparslist,
                              idparsopt = idparsopt,
                              initparsopt = initparsopt,
                              idparsfix = idparsfix,
                              parsfix = initparsfix,
                              sampling_fraction = f,
                              verbose = FALSE,
                              num_threads =  8)
model$k <- 7
saveRDS(model, "SecSSE_Results/Size_results/size_examined_trait_mu_try2.rds")

Warning message in check_ml_conditions(traits, idparslist, initparsopt, idparsopt, :
“Note: you set some transitions as impossible to happen.”


In [28]:
### Try 3
idparsopt <- c(1, 4:9) # our maximum rate parameter was 3
idparsfix <- c(0) # we want to keep all zeros at zero
initparsopt <- c(rep(strt_lambda[3],1), rep(strt_mu[3],3), rep(strt_q[3],3))
initparsfix <- c(0.0) # all zeros remain at zero.
idparslist <- list()
idparslist[[1]] <- lambda_list_cr
idparslist[[2]] <- mu_vec_etd
idparslist[[3]] <- q_matrix

model <- secsse::cla_secsse_ml(phy = phy,
                              traits = traits,
                              num_concealed_states = 3,
                              idparslist = idparslist,
                              idparsopt = idparsopt,
                              initparsopt = initparsopt,
                              idparsfix = idparsfix,
                              parsfix = initparsfix,
                              sampling_fraction = f,
                              verbose = FALSE,
                              num_threads =  8)
model$k <- 7
saveRDS(model, "SecSSE_Results/Size_results/size_examined_trait_mu_try3.rds")

Warning message in check_ml_conditions(traits, idparslist, initparsopt, idparsopt, :
“Note: you set some transitions as impossible to happen.”


### Diversification rates

In [29]:
### Try 1
idparsopt <- c(1:9) # our maximum rate parameter was 3
idparsfix <- c(0) # we want to keep all zeros at zero
initparsopt <- c(rep(strt_lambda[1],3), rep(strt_mu[1],3), rep(strt_q[1],3))
initparsfix <- c(0.0) # all zeros remain at zero.
idparslist <- list()
idparslist[[1]] <- lambda_list_etd
idparslist[[2]] <- mu_vec_etd
idparslist[[3]] <- q_matrix

model <- secsse::cla_secsse_ml(phy = phy,
                              traits = traits,
                              num_concealed_states = 3,
                              idparslist = idparslist,
                              idparsopt = idparsopt,
                              initparsopt = initparsopt,
                              idparsfix = idparsfix,
                              parsfix = initparsfix,
                              sampling_fraction = f,
                              verbose = FALSE,
                              num_threads =  8)
model$k <- 9
saveRDS(model, "SecSSE_Results/Size_results/size_examined_trait_net_diversification_try1.rds")

Warning message in check_ml_conditions(traits, idparslist, initparsopt, idparsopt, :
“Note: you set some transitions as impossible to happen.”


In [30]:
### Try 2
idparsopt <- c(1:9) # our maximum rate parameter was 3
idparsfix <- c(0) # we want to keep all zeros at zero
initparsopt <- c(rep(strt_lambda[2],3), rep(strt_mu[2],3), rep(strt_q[2],3))
initparsfix <- c(0.0) # all zeros remain at zero.
idparslist <- list()
idparslist[[1]] <- lambda_list_etd
idparslist[[2]] <- mu_vec_etd
idparslist[[3]] <- q_matrix

model <- secsse::cla_secsse_ml(phy = phy,
                              traits = traits,
                              num_concealed_states = 3,
                              idparslist = idparslist,
                              idparsopt = idparsopt,
                              initparsopt = initparsopt,
                              idparsfix = idparsfix,
                              parsfix = initparsfix,
                              sampling_fraction = f,
                              verbose = FALSE,
                              num_threads =  8)
model$k <- 9
saveRDS(model, "SecSSE_Results/Size_results/size_examined_trait_net_diversification_try2.rds")

Warning message in check_ml_conditions(traits, idparslist, initparsopt, idparsopt, :
“Note: you set some transitions as impossible to happen.”


In [31]:
### Try 3
idparsopt <- c(1:9) # our maximum rate parameter was 3
idparsfix <- c(0) # we want to keep all zeros at zero
initparsopt <- c(rep(strt_lambda[3],3), rep(strt_mu[3],3), rep(strt_q[3],3))
initparsfix <- c(0.0) # all zeros remain at zero.
idparslist <- list()
idparslist[[1]] <- lambda_list_etd
idparslist[[2]] <- mu_vec_etd
idparslist[[3]] <- q_matrix

model <- secsse::cla_secsse_ml(phy = phy,
                              traits = traits,
                              num_concealed_states = 3,
                              idparslist = idparslist,
                              idparsopt = idparsopt,
                              initparsopt = initparsopt,
                              idparsfix = idparsfix,
                              parsfix = initparsfix,
                              sampling_fraction = f,
                              verbose = FALSE,
                              num_threads =  8)
model$k <- 9
saveRDS(model, "SecSSE_Results/Size_results/size_examined_trait_net_diversification_try3.rds")

Warning message in check_ml_conditions(traits, idparslist, initparsopt, idparsopt, :
“Note: you set some transitions as impossible to happen.”


## Conceal trait

In [32]:
lambda_list_ctd <- secsse::create_lambda_list(state_names = c(0, 1, 2),
                                          num_concealed_states = 3,
                                          transition_matrix = spec_matrix,
                                          model = "CTD")
lambda_list_ctd

,0A,1A,2A,0B,1B,2B,0C,1C,2C
0A,1,0,0,0,0,0,0,0,0
1A,0,0,0,0,0,0,0,0,0
2A,0,0,0,0,0,0,0,0,0
0B,0,0,0,0,0,0,0,0,0
1B,0,0,0,0,0,0,0,0,0
2B,0,0,0,0,0,0,0,0,0
0C,0,0,0,0,0,0,0,0,0
1C,0,0,0,0,0,0,0,0,0
2C,0,0,0,0,0,0,0,0,0
,0A,1A,2A,0B,1B,2B,0C,1C,2C


In [33]:
mu_vec_ctd <- secsse::create_mu_vector(state_names = c(0, 1, 2),
                                   num_concealed_states = 3,
                                   model = "CTD",
                                   lambda_list = lambda_list_ctd)
mu_vec_ctd

0A 1A 2A 0B 1B 2B 0C 1C 2C 
 4  4  4  5  5  5  6  6  6

### Speciation rates

In [34]:
mu_vec_ctd_sp<-rep(4, length(mu_vec_ctd))

In [35]:
### Try 1
idparsopt <- c(1:4, 7:9) # our maximum rate parameter was 3
idparsfix <- c(0) # we want to keep all zeros at zero
initparsopt <- c(rep(strt_lambda[1],3), rep(strt_mu[1],1), rep(strt_q[1],3))
initparsfix <- c(0.0) # all zeros remain at zero.
idparslist <- list()
idparslist[[1]] <- lambda_list_ctd
idparslist[[2]] <- mu_vec_ctd_sp
idparslist[[3]] <- q_matrix

model <- secsse::cla_secsse_ml(phy = phy,
                              traits = traits,
                              num_concealed_states = 3,
                              idparslist = idparslist,
                              idparsopt = idparsopt,
                              initparsopt = initparsopt,
                              idparsfix = idparsfix,
                              parsfix = initparsfix,
                              sampling_fraction = f,
                              verbose = FALSE,
                              num_threads =  8)
model$k <- 7
saveRDS(model, "SecSSE_Results/Size_results/size_concealed_trait_sp_try1.rds")

Warning message in check_ml_conditions(traits, idparslist, initparsopt, idparsopt, :
“Note: you set some transitions as impossible to happen.”


In [37]:
### Try 2
idparsopt <- c(1:4, 7:9) # our maximum rate parameter was 3
idparsfix <- c(0) # we want to keep all zeros at zero
initparsopt <- c(rep(strt_lambda[2],3), rep(strt_mu[2],1), rep(strt_q[2],3))
initparsfix <- c(0.0) # all zeros remain at zero.
idparslist <- list()
idparslist[[1]] <- lambda_list_ctd
idparslist[[2]] <- mu_vec_ctd_sp
idparslist[[3]] <- q_matrix

model <- secsse::cla_secsse_ml(phy = phy,
                              traits = traits,
                              num_concealed_states = 3,
                              idparslist = idparslist,
                              idparsopt = idparsopt,
                              initparsopt = initparsopt,
                              idparsfix = idparsfix,
                              parsfix = initparsfix,
                              sampling_fraction = f,
                              verbose = FALSE,
                              num_threads =  8)
model$k <- 7
saveRDS(model, "SecSSE_Results/Size_results/size_concealed_trait_sp_try2.rds")

Warning message in check_ml_conditions(traits, idparslist, initparsopt, idparsopt, :
“Note: you set some transitions as impossible to happen.”


In [38]:
### Try 3
idparsopt <- c(1:4, 7:9) # our maximum rate parameter was 3
idparsfix <- c(0) # we want to keep all zeros at zero
initparsopt <- c(rep(strt_lambda[3],3), rep(strt_mu[3],1), rep(strt_q[3],3))
initparsfix <- c(0.0) # all zeros remain at zero.
idparslist <- list()
idparslist[[1]] <- lambda_list_ctd
idparslist[[2]] <- mu_vec_ctd_sp
idparslist[[3]] <- q_matrix

model <- secsse::cla_secsse_ml(phy = phy,
                              traits = traits,
                              num_concealed_states = 3,
                              idparslist = idparslist,
                              idparsopt = idparsopt,
                              initparsopt = initparsopt,
                              idparsfix = idparsfix,
                              parsfix = initparsfix,
                              sampling_fraction = f,
                              verbose = FALSE,
                              num_threads =  8)
model$k <- 7
saveRDS(model, "SecSSE_Results/Size_results/size_concealed_trait_sp_try3.rds")

Warning message in check_ml_conditions(traits, idparslist, initparsopt, idparsopt, :
“Note: you set some transitions as impossible to happen.”


### Extinction rates

In [39]:
### Try 1
idparsopt <- c(1, 4:9) # our maximum rate parameter was 3
idparsfix <- c(0) # we want to keep all zeros at zero
initparsopt <- c(rep(strt_lambda[1],1), rep(strt_mu[1],3), rep(strt_q[1],3))
initparsfix <- c(0.0) # all zeros remain at zero.
idparslist <- list()
idparslist[[1]] <- lambda_list_cr
idparslist[[2]] <- mu_vec_ctd
idparslist[[3]] <- q_matrix

model <- secsse::cla_secsse_ml(phy = phy,
                              traits = traits,
                              num_concealed_states = 3,
                              idparslist = idparslist,
                              idparsopt = idparsopt,
                              initparsopt = initparsopt,
                              idparsfix = idparsfix,
                              parsfix = initparsfix,
                              sampling_fraction = f,
                              verbose = FALSE,
                              num_threads =  8)
model$k <- 7
saveRDS(model, "SecSSE_Results/Size_results/size_concealed_trait_mu_try1.rds")

Warning message in check_ml_conditions(traits, idparslist, initparsopt, idparsopt, :
“Note: you set some transitions as impossible to happen.”


In [40]:
### Try 1
idparsopt <- c(1, 4:9) # our maximum rate parameter was 3
idparsfix <- c(0) # we want to keep all zeros at zero
initparsopt <- c(rep(strt_lambda[2],1), rep(strt_mu[2],3), rep(strt_q[2],3))
initparsfix <- c(0.0) # all zeros remain at zero.
idparslist <- list()
idparslist[[1]] <- lambda_list_cr
idparslist[[2]] <- mu_vec_ctd
idparslist[[3]] <- q_matrix

model <- secsse::cla_secsse_ml(phy = phy,
                              traits = traits,
                              num_concealed_states = 3,
                              idparslist = idparslist,
                              idparsopt = idparsopt,
                              initparsopt = initparsopt,
                              idparsfix = idparsfix,
                              parsfix = initparsfix,
                              sampling_fraction = f,
                              verbose = FALSE,
                              num_threads =  8)
model$k <- 7
saveRDS(model, "SecSSE_Results/Size_results/size_concealed_trait_mu_try2.rds")

Warning message in check_ml_conditions(traits, idparslist, initparsopt, idparsopt, :
“Note: you set some transitions as impossible to happen.”


In [41]:
### Try 3
idparsopt <- c(1, 4:9) # our maximum rate parameter was 3
idparsfix <- c(0) # we want to keep all zeros at zero
initparsopt <- c(rep(strt_lambda[3],1), rep(strt_mu[3],3), rep(strt_q[3],3))
initparsfix <- c(0.0) # all zeros remain at zero.
idparslist <- list()
idparslist[[1]] <- lambda_list_cr
idparslist[[2]] <- mu_vec_ctd
idparslist[[3]] <- q_matrix

model <- secsse::cla_secsse_ml(phy = phy,
                              traits = traits,
                              num_concealed_states = 3,
                              idparslist = idparslist,
                              idparsopt = idparsopt,
                              initparsopt = initparsopt,
                              idparsfix = idparsfix,
                              parsfix = initparsfix,
                              sampling_fraction = f,
                              verbose = FALSE,
                              num_threads =  8)
model$k <- 7
saveRDS(model, "SecSSE_Results/Size_results/size_concealed_trait_mu_try3.rds")

Warning message in check_ml_conditions(traits, idparslist, initparsopt, idparsopt, :
“Note: you set some transitions as impossible to happen.”


### Net diversification

In [42]:
### Try 1
idparsopt <- c(1:9) # our maximum rate parameter was 3
idparsfix <- c(0) # we want to keep all zeros at zero
initparsopt <- c(rep(strt_lambda[1],3), rep(strt_mu[1],3), rep(strt_q[1],3))
initparsfix <- c(0.0) # all zeros remain at zero.
idparslist <- list()
idparslist[[1]] <- lambda_list_ctd
idparslist[[2]] <- mu_vec_ctd
idparslist[[3]] <- q_matrix

model <- secsse::cla_secsse_ml(phy = phy,
                              traits = traits,
                              num_concealed_states = 3,
                              idparslist = idparslist,
                              idparsopt = idparsopt,
                              initparsopt = initparsopt,
                              idparsfix = idparsfix,
                              parsfix = initparsfix,
                              sampling_fraction = f,
                              verbose = FALSE,
                              num_threads =  8)
model$k <- 9
saveRDS(model, "SecSSE_Results/Size_results/size_concealed_trait_net_diversification_try1.rds")

Warning message in check_ml_conditions(traits, idparslist, initparsopt, idparsopt, :
“Note: you set some transitions as impossible to happen.”


In [43]:
### Try 2
idparsopt <- c(1:9) # our maximum rate parameter was 3
idparsfix <- c(0) # we want to keep all zeros at zero
initparsopt <- c(rep(strt_lambda[2],3), rep(strt_mu[2],3), rep(strt_q[2],3))
initparsfix <- c(0.0) # all zeros remain at zero.
idparslist <- list()
idparslist[[1]] <- lambda_list_ctd
idparslist[[2]] <- mu_vec_ctd
idparslist[[3]] <- q_matrix

model <- secsse::cla_secsse_ml(phy = phy,
                              traits = traits,
                              num_concealed_states = 3,
                              idparslist = idparslist,
                              idparsopt = idparsopt,
                              initparsopt = initparsopt,
                              idparsfix = idparsfix,
                              parsfix = initparsfix,
                              sampling_fraction = f,
                              verbose = FALSE,
                              num_threads =  8)
model$k <- 9
saveRDS(model, "SecSSE_Results/Size_results/size_concealed_trait_net_diversification_try2.rds")

Warning message in check_ml_conditions(traits, idparslist, initparsopt, idparsopt, :
“Note: you set some transitions as impossible to happen.”


In [44]:
### Try 3
idparsopt <- c(1:9) # our maximum rate parameter was 3
idparsfix <- c(0) # we want to keep all zeros at zero
initparsopt <- c(rep(strt_lambda[3],3), rep(strt_mu[3],3), rep(strt_q[3],3))
initparsfix <- c(0.0) # all zeros remain at zero.
idparslist <- list()
idparslist[[1]] <- lambda_list_ctd
idparslist[[2]] <- mu_vec_ctd
idparslist[[3]] <- q_matrix

model <- secsse::cla_secsse_ml(phy = phy,
                              traits = traits,
                              num_concealed_states = 3,
                              idparslist = idparslist,
                              idparsopt = idparsopt,
                              initparsopt = initparsopt,
                              idparsfix = idparsfix,
                              parsfix = initparsfix,
                              sampling_fraction = f,
                              verbose = FALSE,
                              num_threads =  8)
model$k <- 9
saveRDS(model, "SecSSE_Results/Size_results/size_concealed_trait_net_diversification_try3.rds")

Warning message in check_ml_conditions(traits, idparslist, initparsopt, idparsopt, :
“Note: you set some transitions as impossible to happen.”
